# CH01-02: 必要套件安裝

**課程目標:**
- 掌握 NLP 核心套件的安裝與配置
- 理解不同套件的適用場景
- 學會解決套件衝突問題
- 了解中英文 NLP 工具的差異

**學習時間:** 約 75 分鐘

**前置知識:**
- Poetry 基礎操作
- Python 套件管理概念
- 命令列基本操作

---

## 📚 目錄

1. [NLP 套件生態系統概覽](#1)
2. [中文 NLP 工具安裝](#2)
3. [英文 NLP 工具安裝](#3)
4. [深度學習框架選擇](#4)
5. [Transformers 生態安裝](#5)
6. [套件相容性檢查](#6)
7. [模型與數據下載](#7)
8. [實戰練習](#8)

---

In [ ]:
# 環境設定與套件導入
import subprocess
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from importlib.metadata import version, PackageNotFoundError
from pathlib import Path

# 設定中文顯示
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 設定顯示風格
sns.set_style('whitegrid')

print("✅ 環境設定完成")
print(f"Python 版本: {sys.version.split()[0]}")

<a id="1"></a>
## 1. NLP 套件生態系統概覽

### 1.1 NLP 工具分類

NLP 套件可從**語言**、**技術典範**、**應用場景**三個維度分類:

#### 按語言分類:
- **中文專用:** jieba, pkuseg, LTP
- **英文專用:** NLTK, TextBlob
- **多語言:** spaCy, Stanza

#### 按技術典範分類:
- **傳統方法 (規則/統計):** NLTK, jieba
- **深度學習:** Transformers, fairseq

#### 按應用場景分類:
- **研究教學:** NLTK
- **生產環境:** spaCy, Transformers

---

In [ ]:
# NLP 套件生態系統視覺化
packages_data = {
    '套件': ['jieba', 'NLTK', 'spaCy', 'Transformers', 'Stanza', 'Gensim'],
    '適用語言': ['中文', '英文', '多語言', '多語言', '60+ 語言', '多語言'],
    '主要功能': ['分詞', '分詞/POS/詞幹化', 'Pipeline 處理', '預訓練模型', '分詞/POS/NER', 'Word2Vec/Doc2Vec'],
    '技術典範': ['統計 (HMM)', '規則+統計', '神經網路', 'Transformer', '神經網路', '詞向量'],
    '學習曲線': ['低', '中', '中', '高', '中', '中'],
    '效能': ['快', '中', '快', '慢 (需 GPU)', '中', '中']
}

df_packages = pd.DataFrame(packages_data)

fig, ax = plt.subplots(figsize=(16, 6))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_packages.values, colLabels=df_packages.columns,
                cellLoc='center', loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# 設定表頭樣式
for i in range(len(df_packages.columns)):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 設定交替行顏色
for i in range(1, len(df_packages) + 1):
    for j in range(len(df_packages.columns)):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#F7F7F7')

plt.title('NLP 套件生態系統對比', fontsize=18, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📊 選擇指南:")
print("  - 中文分詞: jieba (快速、輕量)")
print("  - 英文教學: NLTK (功能完整)")
print("  - 生產環境: spaCy (效能優先)")
print("  - SOTA 模型: Transformers (準確率最高)")
print("  - 多語言: Stanza (支援 60+ 語言)")

### 1.2 套件安裝策略

#### 分層安裝策略 (推薦):

```bash
# 第 1 層: 基礎工具
poetry add numpy pandas scikit-learn matplotlib

# 第 2 層: 中文/英文 NLP
poetry add jieba nltk spacy

# 第 3 層: 深度學習框架 (二選一)
poetry add torch  # 或 tensorflow

# 第 4 層: 深度學習 NLP
poetry add transformers datasets tokenizers
```

**優點:**
- 分層安裝,問題易定位
- 避免一次安裝過多套件導致衝突
- 便於理解依賴關係

---

<a id="2"></a>
## 2. 中文 NLP 工具安裝

### 2.1 jieba (結巴分詞)

**jieba** 是最流行的 Python 中文分詞工具,基於 HMM (隱馬可夫模型) 與 Viterbi 算法。

#### 安裝與配置:

In [ ]:
# 安裝 jieba (在終端執行)
install_cmd = "poetry add jieba"
print(f"📦 安裝指令: {install_cmd}\n")

# 檢查是否已安裝
try:
    import jieba
    jieba_version = version('jieba')
    print(f"✅ jieba 已安裝,版本: {jieba_version}")
    
    # 基本測試
    test_text = "我愛自然語言處理"
    words = list(jieba.cut(test_text))
    print(f"\n分詞測試: '{test_text}' → {words}")
    
except ImportError:
    print(f"❌ jieba 未安裝,請執行: {install_cmd}")
except PackageNotFoundError:
    print(f"⚠️ jieba 已導入但版本資訊不可用")

#### jieba 三種分詞模式:

In [ ]:
# jieba 分詞模式對比
try:
    import jieba
    
    test_sentence = "我來到北京清華大學"
    
    modes = [
        ('精確模式', list(jieba.cut(test_sentence, cut_all=False))),
        ('全模式', list(jieba.cut(test_sentence, cut_all=True))),
        ('搜尋引擎模式', list(jieba.cut_for_search(test_sentence)))
    ]
    
    print(f"測試句子: '{test_sentence}'\n")
    
    for mode_name, result in modes:
        print(f"{mode_name:12s}: {result}")
    
    # 視覺化對比
    mode_names = [m[0] for m in modes]
    token_counts = [len(m[1]) for m in modes]
    
    plt.figure(figsize=(10, 5))
    bars = plt.bar(mode_names, token_counts, color=['#4ECDC4', '#FF6B6B', '#FFE66D'])
    plt.ylabel('詞數量', fontsize=12)
    plt.title('jieba 三種分詞模式對比', fontsize=14, fontweight='bold')
    
    # 在柱狀圖上標註數值
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 模式選擇:")
    print("  - 精確模式: 適合文本分析 (預設)")
    print("  - 全模式: 窮盡所有可能,詞數最多")
    print("  - 搜尋引擎模式: 適合建立索引")
    
except ImportError:
    print("請先安裝 jieba: poetry add jieba")

### 2.2 自定義詞典

jieba 支援自定義詞典,可添加領域專有名詞:

In [ ]:
# 自定義詞典示例
try:
    import jieba
    
    # 測試文本 (包含專有名詞)
    text = "我在學習 Transformer 模型和 BERT 算法"
    
    print("測試文本:", text)
    print("\n【未加載自定義詞典】")
    result_before = list(jieba.cut(text))
    print(f"分詞結果: {result_before}")
    
    # 添加自定義詞
    custom_words = [
        ('Transformer', 10, 'n'),  # (詞, 詞頻, 詞性)
        ('BERT', 10, 'n'),
        ('自然語言處理', 15, 'n')
    ]
    
    for word, freq, pos in custom_words:
        jieba.add_word(word, freq, pos)
    
    print("\n【加載自定義詞典後】")
    result_after = list(jieba.cut(text))
    print(f"分詞結果: {result_after}")
    
    # 對比視覺化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 未加載詞典
    axes[0].bar(range(len(result_before)), [1]*len(result_before), color='#FF6B6B')
    axes[0].set_xticks(range(len(result_before)))
    axes[0].set_xticklabels(result_before, rotation=45, ha='right')
    axes[0].set_title('未加載自定義詞典', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('詞位置')
    
    # 加載詞典後
    axes[1].bar(range(len(result_after)), [1]*len(result_after), color='#4ECDC4')
    axes[1].set_xticks(range(len(result_after)))
    axes[1].set_xticklabels(result_after, rotation=45, ha='right')
    axes[1].set_title('加載自定義詞典後', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('詞位置')
    
    plt.suptitle('自定義詞典效果對比', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 自定義詞典格式 (custom_dict.txt):")
    print("  Transformer 10 n")
    print("  BERT 10 n")
    print("  自然語言處理 15 n")
    print("\n載入方式: jieba.load_userdict('custom_dict.txt')")
    
except ImportError:
    print("請先安裝 jieba")

<a id="3"></a>
## 3. 英文 NLP 工具安裝

### 3.1 NLTK (Natural Language Toolkit)

**NLTK** 是經典的英文 NLP 工具包,適合教學與研究。

#### 安裝與數據下載:

In [ ]:
# 安裝 NLTK
install_cmd = "poetry add nltk"
print(f"📦 安裝指令: {install_cmd}\n")

# 檢查並測試 NLTK
try:
    import nltk
    nltk_version = version('nltk')
    print(f"✅ NLTK 已安裝,版本: {nltk_version}")
    
    # 必要數據包列表
    required_data = [
        ('punkt', '分詞模型'),
        ('stopwords', '停用詞表'),
        ('averaged_perceptron_tagger', 'POS 標註器'),
        ('wordnet', 'WordNet 詞彙資料庫'),
        ('omw-1.4', '多語言 WordNet')
    ]
    
    print("\n📚 必要數據包:")
    for package, description in required_data:
        print(f"  - {package:30s}: {description}")
    
    print("\n下載指令:")
    print("  nltk.download('punkt')")
    print("  nltk.download('stopwords')")
    print("  nltk.download('averaged_perceptron_tagger')")
    print("  nltk.download('wordnet')")
    print("  nltk.download('omw-1.4')")
    print("\n或一次性下載: nltk.download('popular')")
    
except ImportError:
    print(f"❌ NLTK 未安裝,請執行: {install_cmd}")

In [ ]:
# NLTK 基本功能測試
try:
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    
    # 測試文本
    text = "Natural language processing is amazing! It enables computers to understand human language."
    
    print(f"測試文本: {text}\n")
    
    # 分詞
    try:
        tokens = word_tokenize(text)
        print(f"✅ 分詞結果: {tokens}")
        print(f"   詞數: {len(tokens)}")
    except LookupError:
        print("❌ 分詞失敗,請下載: nltk.download('punkt')")
        tokens = []
    
    # 停用詞過濾
    try:
        stop_words = set(stopwords.words('english'))
        filtered = [w for w in tokens if w.lower() not in stop_words]
        print(f"\n✅ 過濾停用詞後: {filtered}")
        print(f"   詞數: {len(filtered)}")
        
        # 視覺化對比
        data = {
            '處理階段': ['原始分詞', '過濾停用詞'],
            '詞數': [len(tokens), len(filtered)]
        }
        
        plt.figure(figsize=(10, 5))
        bars = plt.bar(data['處理階段'], data['詞數'], color=['#FF6B6B', '#4ECDC4'])
        plt.ylabel('詞數量', fontsize=12)
        plt.title('NLTK 停用詞過濾效果', fontsize=14, fontweight='bold')
        
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}',
                    ha='center', va='bottom', fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
    except LookupError:
        print("❌ 停用詞處理失敗,請下載: nltk.download('stopwords')")
    
except ImportError:
    print("請先安裝 NLTK: poetry add nltk")

### 3.2 spaCy (工業級 NLP)

**spaCy** 是面向生產環境的 NLP 工具,效能優於 NLTK。

#### 安裝與語言模型下載:

In [ ]:
# 安裝 spaCy
install_steps = [
    ('1. 安裝 spaCy 套件', 'poetry add spacy'),
    ('2. 下載英文小型模型', 'python -m spacy download en_core_web_sm'),
    ('3. 下載英文中型模型 (含詞向量)', 'python -m spacy download en_core_web_md'),
    ('4. 下載中文模型', 'python -m spacy download zh_core_web_sm'),
]

print("📦 spaCy 安裝步驟:\n")
for step, cmd in install_steps:
    print(f"{step}")
    print(f"   {cmd}\n")

# 檢查 spaCy 安裝狀態
try:
    import spacy
    spacy_version = version('spacy')
    print(f"✅ spaCy 已安裝,版本: {spacy_version}")
    
    # 檢查可用模型
    print("\n🔍 檢查已安裝的語言模型:")
    models = ['en_core_web_sm', 'en_core_web_md', 'zh_core_web_sm']
    
    for model in models:
        try:
            nlp = spacy.load(model)
            print(f"  ✅ {model} - 已安裝")
        except OSError:
            print(f"  ❌ {model} - 未安裝")
            print(f"     下載: python -m spacy download {model}")
    
except ImportError:
    print("❌ spaCy 未安裝,請執行: poetry add spacy")

In [ ]:
# spaCy 功能展示
try:
    import spacy
    
    # 載入模型
    try:
        nlp = spacy.load('en_core_web_sm')
        
        # 處理文本
        text = "Apple is looking at buying U.K. startup for $1 billion"
        doc = nlp(text)
        
        print(f"測試文本: {text}\n")
        
        # 實體識別 (NER)
        print("【命名實體識別 (NER)】")
        for ent in doc.ents:
            print(f"  {ent.text:15s} → {ent.label_:10s} ({spacy.explain(ent.label_)})")
        
        # 詞性標註 (POS)
        print("\n【詞性標註 (POS)】")
        for token in doc:
            print(f"  {token.text:10s} → {token.pos_:6s} {token.dep_:10s}")
        
        # 視覺化 NER 結果
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        if entities:
            ent_texts, ent_labels = zip(*entities)
            
            plt.figure(figsize=(12, 5))
            colors = {'ORG': '#FF6B6B', 'GPE': '#4ECDC4', 'MONEY': '#FFE66D'}
            bar_colors = [colors.get(label, '#95E1D3') for label in ent_labels]
            
            plt.bar(range(len(ent_texts)), [1]*len(ent_texts), color=bar_colors)
            plt.xticks(range(len(ent_texts)), ent_texts, rotation=45, ha='right')
            plt.ylabel('實體')
            plt.title('spaCy 命名實體識別結果', fontsize=14, fontweight='bold')
            
            # 圖例
            from matplotlib.patches import Patch
            legend_elements = [Patch(facecolor=colors[label], label=label) for label in set(ent_labels)]
            plt.legend(handles=legend_elements, loc='upper right')
            
            plt.tight_layout()
            plt.show()
        
    except OSError:
        print("❌ 英文模型未安裝")
        print("請執行: python -m spacy download en_core_web_sm")
    
except ImportError:
    print("請先安裝 spaCy: poetry add spacy")

<a id="4"></a>
## 4. 深度學習框架選擇

### 4.1 PyTorch vs TensorFlow

深度學習 NLP 需要選擇底層框架,主流選項為 **PyTorch** 和 **TensorFlow**。

#### 對比分析:

In [ ]:
# PyTorch vs TensorFlow 對比
comparison = {
    '維度': ['學習曲線', '社群支援', 'NLP 生態', '生產部署', '研究友善度', 'GPU 支援', '模型可用性'],
    'PyTorch': ['中等', '⭐⭐⭐⭐⭐', 'Transformers 首選', '需額外工具', '⭐⭐⭐⭐⭐', '優秀', 'Hugging Face'],
    'TensorFlow': ['較陡', '⭐⭐⭐⭐', 'TF Hub', 'TF Serving 完善', '⭐⭐⭐', '優秀', 'TF Hub']
}

df_compare = pd.DataFrame(comparison)

fig, ax = plt.subplots(figsize=(14, 6))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_compare.values, colLabels=df_compare.columns,
                cellLoc='center', loc='center',
                colWidths=[0.25, 0.35, 0.35])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# 設定表頭
for i in range(len(df_compare.columns)):
    table[(0, i)].set_facecolor('#FF6B6B')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 高亮 PyTorch 列
for i in range(1, len(df_compare) + 1):
    table[(i, 1)].set_facecolor('#FFE6E6')

plt.title('PyTorch vs TensorFlow 對比', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n🎯 選擇建議:")
print("\nPyTorch - 適合:")
print("  ✅ NLP 研究與實驗")
print("  ✅ 使用 Hugging Face Transformers")
print("  ✅ 需要動態計算圖")
print("  ✅ 學術論文復現")

print("\nTensorFlow - 適合:")
print("  ✅ 大規模生產部署")
print("  ✅ 移動端/嵌入式設備 (TF Lite)")
print("  ✅ 企業級應用")
print("  ✅ 需要完整部署方案 (TF Serving)")

print("\n💡 本課程推薦: PyTorch")
print("   原因: Transformers 生態、社群活躍、學習資源豐富")

### 4.2 安裝深度學習框架

#### PyTorch 安裝:

In [ ]:
# PyTorch 安裝指令
pytorch_install = {
    '環境': ['CPU 版本', 'GPU 版本 (CUDA 11.8)', 'GPU 版本 (CUDA 12.1)', 'macOS (M1/M2)'],
    '安裝指令': [
        'poetry add torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu',
        'poetry add torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118',
        'poetry add torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121',
        'poetry add torch torchvision torchaudio'
    ]
}

print("🔥 PyTorch 安裝指令:\n")
for env, cmd in zip(pytorch_install['環境'], pytorch_install['安裝指令']):
    print(f"{env}:")
    print(f"  {cmd}\n")

print("⚠️ 注意事項:")
print("  1. 先確認 CUDA 版本: nvidia-smi")
print("  2. 選擇對應的 PyTorch 版本")
print("  3. CPU 版本體積較小,但無法使用 GPU 加速")

# 檢查 PyTorch 安裝
try:
    import torch
    torch_version = version('torch')
    print(f"\n✅ PyTorch 已安裝,版本: {torch_version}")
    print(f"   CUDA 可用: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   CUDA 版本: {torch.version.cuda}")
        print(f"   GPU 數量: {torch.cuda.device_count()}")
        print(f"   GPU 名稱: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("\n❌ PyTorch 未安裝")

#### TensorFlow 安裝 (可選):

In [ ]:
# TensorFlow 安裝指令
print("🔷 TensorFlow 安裝指令:\n")
print("CPU 版本:")
print("  poetry add tensorflow\n")
print("GPU 版本:")
print("  poetry add tensorflow[and-cuda]\n")

# 檢查 TensorFlow 安裝
try:
    import tensorflow as tf
    tf_version = version('tensorflow')
    print(f"✅ TensorFlow 已安裝,版本: {tf_version}")
    
    gpus = tf.config.list_physical_devices('GPU')
    print(f"   GPU 數量: {len(gpus)}")
    for gpu in gpus:
        print(f"   - {gpu}")
except ImportError:
    print("❌ TensorFlow 未安裝 (可選)")

<a id="5"></a>
## 5. Transformers 生態安裝

### 5.1 Hugging Face Transformers

**Transformers** 是目前最流行的預訓練模型庫,支援 BERT、GPT、T5 等數千個模型。

#### 核心套件:

In [ ]:
# Transformers 生態套件
transformers_packages = {
    '套件': ['transformers', 'datasets', 'tokenizers', 'accelerate', 'evaluate'],
    '功能': [
        '預訓練模型庫 (BERT/GPT/T5)',
        '數據集載入與處理',
        '快速 Tokenizer (Rust 實作)',
        '多 GPU/混合精度訓練',
        '模型評估指標'
    ],
    '必要性': ['必須', '推薦', '推薦', '進階', '進階'],
    '安裝指令': [
        'poetry add transformers',
        'poetry add datasets',
        'poetry add tokenizers',
        'poetry add accelerate',
        'poetry add evaluate'
    ]
}

df_transformers = pd.DataFrame(transformers_packages)

fig, ax = plt.subplots(figsize=(16, 5))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_transformers.values, colLabels=df_transformers.columns,
                cellLoc='left', loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# 設定表頭
for i in range(len(df_transformers.columns)):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 高亮必須套件
table[(1, 2)].set_facecolor('#FFEB3B')  # transformers

plt.title('Transformers 生態套件', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📦 安裝順序建議:")
print("  1. poetry add transformers")
print("  2. poetry add datasets tokenizers")
print("  3. poetry add accelerate evaluate (進階功能)")

In [ ]:
# 檢查 Transformers 安裝
transformers_check = [
    ('transformers', '預訓練模型'),
    ('datasets', '數據集'),
    ('tokenizers', 'Tokenizer')
]

print("🔍 檢查 Transformers 生態套件安裝狀態:\n")

for package, desc in transformers_check:
    try:
        pkg_version = version(package)
        print(f"  ✅ {package:15s} {pkg_version:10s} - {desc}")
    except PackageNotFoundError:
        print(f"  ❌ {package:15s} {'未安裝':10s} - {desc}")
        print(f"     安裝: poetry add {package}")

# 測試 Transformers
try:
    from transformers import AutoTokenizer, AutoModel
    print("\n✅ Transformers 核心模組可正常導入")
    print("\n💡 使用範例:")
    print("  from transformers import AutoTokenizer, AutoModel")
    print("  tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')")
    print("  model = AutoModel.from_pretrained('bert-base-chinese')")
except ImportError:
    print("\n❌ Transformers 未安裝或導入失敗")

### 5.2 模型快取設定

Transformers 模型檔案通常較大 (數百 MB),需要合理設定快取目錄:

In [ ]:
# Transformers 快取設定
import os

print("📁 Transformers 快取目錄設定:\n")

# 預設快取位置
default_cache = os.path.expanduser('~/.cache/huggingface/hub/')
print(f"預設快取位置: {default_cache}")

print("\n自定義快取目錄 (三種方法):\n")

print("方法 1: 環境變數 (全域設定)")
print("  export TRANSFORMERS_CACHE=/path/to/cache  # Linux/macOS")
print("  set TRANSFORMERS_CACHE=D:\\models         # Windows\n")

print("方法 2: Python 代碼設定")
print("  import os")
print("  os.environ['TRANSFORMERS_CACHE'] = '/path/to/cache'\n")

print("方法 3: 下載時指定")
print("  model = AutoModel.from_pretrained('bert-base-chinese',")
print("                                     cache_dir='/path/to/cache')")

# 視覺化快取目錄結構
cache_structure = """
~/.cache/huggingface/
└── hub/
    ├── models--bert-base-chinese/
    │   ├── snapshots/
    │   │   └── abc123.../
    │   │       ├── config.json
    │   │       ├── pytorch_model.bin (約 400MB)
    │   │       └── tokenizer.json
    │   └── refs/
    │       └── main → abc123...
    └── models--gpt2/
        └── ...
"""

print("\n📂 快取目錄結構:")
print(cache_structure)

print("\n💡 快取管理建議:")
print("  1. 定期清理不用的模型 (可手動刪除)")
print("  2. 團隊共享可設定統一快取目錄")
print("  3. CI/CD 環境建議使用快取加速")

<a id="6"></a>
## 6. 套件相容性檢查

### 6.1 常見依賴衝突

深度學習 NLP 專案常見的套件衝突問題:

In [ ]:
# 常見依賴衝突案例
conflicts = {
    '衝突類型': [
        'NumPy 版本衝突',
        'Protobuf 版本衝突',
        'PyTorch vs TensorFlow',
        'Tokenizers 版本不匹配'
    ],
    '症狀': [
        'ImportError: numpy.core.multiarray failed',
        'TypeError: Descriptors cannot not be created directly',
        '記憶體衝突,無法同時載入',
        'AttributeError: module has no attribute'
    ],
    '解決方案': [
        'poetry add "numpy>=1.24,<2.0"',
        'poetry add "protobuf>=3.20,<4.0"',
        '選擇一個框架,避免同時安裝',
        'poetry update tokenizers transformers'
    ]
}

df_conflicts = pd.DataFrame(conflicts)

fig, ax = plt.subplots(figsize=(16, 5))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_conflicts.values, colLabels=df_conflicts.columns,
                cellLoc='left', loc='center')

table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 3)

# 設定表頭
for i in range(len(df_conflicts.columns)):
    table[(0, i)].set_facecolor('#FF6B6B')
    table[(0, i)].set_text_props(weight='bold', color='white')

plt.title('常見依賴衝突與解決方案', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n🔧 預防衝突的最佳實踐:")
print("  1. 使用 Poetry 管理依賴 - 自動解決衝突")
print("  2. 定期更新 poetry.lock - poetry update")
print("  3. 檢查依賴樹 - poetry show --tree")
print("  4. 鎖定核心套件版本範圍 - 使用 ^ 或 ~ 約束")

### 6.2 環境驗證腳本

創建自動化腳本檢查所有套件:

In [ ]:
# 完整環境驗證腳本
def check_nlp_environment():
    """檢查 NLP 開發環境所有套件"""
    
    packages = [
        # 基礎工具
        ('numpy', '1.24.0'),
        ('pandas', '2.0.0'),
        ('scikit-learn', '1.3.0'),
        # 中文 NLP
        ('jieba', None),
        # 英文 NLP
        ('nltk', '3.8.0'),
        ('spacy', '3.7.0'),
        # 深度學習
        ('torch', '2.0.0'),
        ('transformers', '4.30.0'),
        ('datasets', None),
        ('tokenizers', None),
    ]
    
    print("="*70)
    print("  NLP 環境套件檢查報告")
    print("="*70)
    
    results = []
    
    for package, min_version in packages:
        try:
            pkg_version = version(package)
            status = "✅"
            
            if min_version and pkg_version < min_version:
                status = "⚠️"
                note = f"建議 >= {min_version}"
            else:
                note = "OK"
            
            results.append({
                '套件': package,
                '版本': pkg_version,
                '狀態': status,
                '備註': note
            })
            
        except PackageNotFoundError:
            results.append({
                '套件': package,
                '版本': '未安裝',
                '狀態': '❌',
                '備註': f'執行: poetry add {package}'
            })
    
    # 輸出結果
    for r in results:
        print(f"  {r['狀態']} {r['套件']:20s} {r['版本']:15s} {r['備註']}")
    
    print("\n" + "="*70)
    
    # 統計
    installed = sum(1 for r in results if r['狀態'] == '✅')
    total = len(results)
    
    print(f"  套件安裝狀態: {installed}/{total} 已安裝")
    
    if installed == total:
        print("\n  ✅ 環境檢查通過,可以開始 NLP 開發!")
    else:
        print("\n  ⚠️ 部分套件缺失或版本過低,請根據上方提示安裝")
    
    print("="*70)
    
    return results

# 執行檢查
results = check_nlp_environment()

<a id="7"></a>
## 7. 模型與數據下載

### 7.1 NLTK 數據下載

NLTK 需要額外下載語料庫與模型:

In [ ]:
# NLTK 數據下載腳本
def download_nltk_data():
    """批次下載 NLTK 必要數據"""
    try:
        import nltk
        
        data_packages = [
            'punkt',
            'stopwords',
            'averaged_perceptron_tagger',
            'wordnet',
            'omw-1.4'
        ]
        
        print("📥 開始下載 NLTK 數據包...\n")
        
        for package in data_packages:
            try:
                # 檢查是否已下載
                nltk.data.find(f'tokenizers/{package}' if package == 'punkt' else f'corpora/{package}')
                print(f"  ✅ {package:30s} - 已存在")
            except LookupError:
                print(f"  ⬇️  {package:30s} - 下載中...")
                nltk.download(package, quiet=True)
                print(f"  ✅ {package:30s} - 下載完成")
        
        print("\n✅ NLTK 數據包下載完成!")
        
    except ImportError:
        print("❌ NLTK 未安裝,請先執行: poetry add nltk")

# 執行下載 (取消註解以執行)
# download_nltk_data()

print("💡 使用說明:")
print("  - 取消上方註解執行自動下載")
print("  - 或手動執行: nltk.download('popular')")
print("  - 或使用 GUI: nltk.download()")

### 7.2 spaCy 語言模型

spaCy 模型大小與功能對比:

In [ ]:
# spaCy 模型對比
spacy_models = {
    '模型': [
        'en_core_web_sm',
        'en_core_web_md',
        'en_core_web_lg',
        'zh_core_web_sm',
        'zh_core_web_md'
    ],
    '語言': ['英文', '英文', '英文', '中文', '中文'],
    '大小': ['12 MB', '40 MB', '560 MB', '40 MB', '80 MB'],
    '詞向量': ['❌', '✅ (300 維)', '✅ (300 維)', '❌', '✅'],
    '功能': [
        'POS, NER, Parser',
        'POS, NER, Parser, Vectors',
        'POS, NER, Parser, Vectors',
        'POS, NER, Parser',
        'POS, NER, Parser, Vectors'
    ],
    '推薦場景': [
        '快速開發/測試',
        '一般應用',
        '高準確率需求',
        '中文基本處理',
        '中文語義分析'
    ]
}

df_spacy = pd.DataFrame(spacy_models)

fig, ax = plt.subplots(figsize=(16, 6))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=df_spacy.values, colLabels=df_spacy.columns,
                cellLoc='center', loc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# 設定表頭
for i in range(len(df_spacy.columns)):
    table[(0, i)].set_facecolor('#4ECDC4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# 高亮推薦模型
table[(2, 0)].set_facecolor('#FFEB3B')  # en_core_web_md
table[(4, 0)].set_facecolor('#FFEB3B')  # zh_core_web_sm

plt.title('spaCy 語言模型對比', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📥 下載指令:")
print("  python -m spacy download en_core_web_md  # 推薦 (英文)")
print("  python -m spacy download zh_core_web_sm  # 推薦 (中文)")

print("\n💡 選擇建議:")
print("  - 開發測試: 使用 sm (小型) 模型")
print("  - 生產環境: 使用 md (中型) 模型 (含詞向量)")
print("  - 高準確率: 使用 lg (大型) 模型 (需更多記憶體)")

<a id="8"></a>
## 8. 實戰練習

### 練習 1: 完整環境搭建

根據以下步驟,從零搭建完整 NLP 環境:

In [ ]:
# 練習 1: 完整環境搭建步驟
exercise1 = """
📝 練習 1: 從零搭建 NLP 環境

步驟 1: 創建專案
  poetry new nlp-complete-env
  cd nlp-complete-env
  poetry config virtualenvs.in-project true

步驟 2: 安裝基礎工具
  poetry add numpy pandas scikit-learn matplotlib seaborn

步驟 3: 安裝中文 NLP
  poetry add jieba

步驟 4: 安裝英文 NLP
  poetry add nltk spacy
  python -m spacy download en_core_web_md

步驟 5: 安裝深度學習框架
  poetry add torch torchvision torchaudio

步驟 6: 安裝 Transformers 生態
  poetry add transformers datasets tokenizers

步驟 7: 下載 NLTK 數據
  python -c "import nltk; nltk.download('popular')"

步驟 8: 環境驗證
  poetry run python -c "import jieba, nltk, spacy, torch, transformers; print('✅ All OK')"

步驟 9: 檢查依賴樹
  poetry show --tree

步驟 10: 提交配置
  git init
  git add pyproject.toml poetry.lock
  git commit -m "feat: initial NLP environment setup"
"""

print(exercise1)

print("\n🎯 預期結果:")
print("  - 所有套件正確安裝")
print("  - 依賴無衝突")
print("  - 語言模型已下載")
print("  - 可正常導入所有套件")

### 練習 2: 套件功能測試

測試所有已安裝套件的核心功能:

In [ ]:
# 練習 2: 綜合功能測試
def test_all_packages():
    """測試所有 NLP 套件核心功能"""
    
    print("🧪 NLP 套件功能測試\n")
    print("="*70)
    
    # 測試 1: jieba
    try:
        import jieba
        result = list(jieba.cut("我愛自然語言處理"))
        print(f"✅ jieba 分詞: {result}")
    except:
        print("❌ jieba 測試失敗")
    
    # 測試 2: NLTK
    try:
        from nltk.tokenize import word_tokenize
        result = word_tokenize("NLP is awesome")
        print(f"✅ NLTK 分詞: {result}")
    except:
        print("❌ NLTK 測試失敗 (可能缺少數據包)")
    
    # 測試 3: spaCy
    try:
        import spacy
        nlp = spacy.load('en_core_web_sm')
        doc = nlp("Apple is a company")
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        print(f"✅ spaCy NER: {entities}")
    except:
        print("❌ spaCy 測試失敗 (可能缺少模型)")
    
    # 測試 4: PyTorch
    try:
        import torch
        x = torch.tensor([1, 2, 3])
        print(f"✅ PyTorch: {x}, CUDA={torch.cuda.is_available()}")
    except:
        print("❌ PyTorch 測試失敗")
    
    # 測試 5: Transformers
    try:
        from transformers import AutoTokenizer
        print(f"✅ Transformers: 模組載入成功")
    except:
        print("❌ Transformers 測試失敗")
    
    print("="*70)
    print("\n測試完成!")

# 執行測試 (取消註解)
# test_all_packages()

print("💡 取消上方註解執行完整測試")

---

## 📚 本課總結

### 核心要點回顧:

1. **NLP 套件分類:**
   - 中文專用: jieba (輕量、快速)
   - 英文工具: NLTK (教學)、spaCy (生產)
   - 深度學習: Transformers (SOTA 模型)

2. **安裝策略:**
   - 分層安裝 (基礎 → NLP → 深度學習)
   - 使用 Poetry 管理依賴
   - 定期檢查相容性

3. **深度學習框架:**
   - PyTorch: NLP 研究首選,Transformers 生態
   - TensorFlow: 生產部署完善

4. **模型管理:**
   - NLTK: 需下載數據包 (punkt, stopwords 等)
   - spaCy: 需下載語言模型 (en_core_web_md 等)
   - Transformers: 自動快取模型 (~/.cache/huggingface)

5. **相容性檢查:**
   - 使用環境驗證腳本
   - 定期執行 poetry update
   - 檢查依賴樹 (poetry show --tree)

---

## 🎯 下節預告

**CH01-03: 開發環境測試**

我們將深入探討:
- 完整環境檢測腳本
- GPU 環境測試 (CUDA, cuDNN)
- 模型載入驗證
- 效能基準測試
- 常見問題診斷與排除

---

## 📖 延伸閱讀

1. **官方文檔:**
   - [jieba GitHub](https://github.com/fxsjy/jieba)
   - [NLTK 官方文檔](https://www.nltk.org/)
   - [spaCy 官方文檔](https://spacy.io/)
   - [Transformers 官方文檔](https://huggingface.co/docs/transformers/)

2. **模型資源:**
   - [Hugging Face Hub](https://huggingface.co/models)
   - [spaCy 模型列表](https://spacy.io/models)
   - [NLTK 數據下載](https://www.nltk.org/data.html)

3. **進階主題:**
   - [中文分詞算法詳解](https://github.com/hankcs/HanLP)
   - [spaCy 進階教程](https://course.spacy.io/)
   - [Transformers 課程](https://huggingface.co/course/)

---

### 🙋 問題討論

有任何問題嗎?歡迎在討論區提問!

---

**課程資訊:**
- **作者:** iSpan NLP Team
- **版本:** v1.0
- **最後更新:** 2025-10-17
- **授權:** MIT License (僅供教學使用)